In [47]:
import os

import warnings
warnings.filterwarnings("ignore")

with open('../openai_api_key.txt', 'r') as f:
    os.environ["OPENAI_API_KEY"] = f.read()

In [48]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from datetime import date
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.chains import SequentialChain, RouterChain, ConversationChain, LLMRouterChain, MultiPromptChain, TransformChain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser


In [49]:
llm = ChatOpenAI(temperature=0)

In [50]:
personalData = """\
    city: London,\
    job-type: [Data Scientist, AI Software Engineer],\
    monthly-earnings: [1500 usd, 8000 usd],\
    skills: [fast learning, reading complicated documentation,\ 
        creating clean data representation, data analysis],\
    interests: [generating ai images, data analysis, large language models],\
    benefits: [flexible hours, remote, health-insurance],\
    technology: [linux -medium, python -advanced, pytorch -advanced, matlabplot -beginner,\
    stable diffusion -beginner, vsc -advanced],\
    languages: [English C1, Polish C1]
    """

In [51]:
# jobOffer = "C++ Backend Developer Volue Sp. z o.o. technology image Category: Backend , C++ Mid Must have C++English (B2) Nice to have PythonC#Polish Requirements description C++ programming experience and knowledge of modern C++ Familiarity with Microsoft platform and tools is beneficial Experience working in larger C++ codebases Experience working effectively in a distributed team Some experience with using databases Some knowledge of Python and C# Offer description Volue is a market leader in technologies and services that power the green transition. Around 800 employees work with more than 2 200 customers on energy, power grid, water & infrastructure projects.Our mission is to provide innovative services critical to society, unlocking a cleaner, better and more profitable future. Working towards this goal, Volue has become a leading technology supplier and enabler of the green transition, helping energy companies simplify and optimize everyday operations.We are now looking for Mid/Senior Software Engineer (Mesh Team), who can help us to continuously improve Volue’s ability to provide value to customers in line with the Volue mission. Volue Mesh is a data management system based on object modelling with an integrated time series calculation engine. It is used for more efficient managing of time series data and related information used in different work processes and serves as the core of all future energy production planning and trading solutions.read more Your responsibilities Contribute to all development phases: design, develop, build, deploy and maintain the C++ codebase of Volue Mesh Work closely together with architects, domain experts and backend developers Create and maintain user friendly solutions with clear APIs with efficient resource usage and high performance Work in an international environment, where English is the spoken and written language. The team is distributed in Gdańsk and Trondheim Work in a distributed team with a flexible work-from-home policy Work closely with data intensive applications that are using Mesh and Time Series Management systems to simplify complex workflows show all (7) Job details Online recruitment Recruitment language: Polish&English Start ASAP Permanent contract Remote days: flexible in a week Flexible hours Relocation package: 6000 PLN Perks in the office Free snacks Free coffee Free beverages Gym Canteen Bike parking Playroom Shower No dress code Benefits Sport subscription Life & group insurance Private healthcare English classes Training budget Flat structure Small teams International projects"
jobOffer = """city: London,\
    job-type: [Data Scientist, AI Software Engineer],\
    monthly-earnings: [1500 usd, 8000 usd],\
    skills: [fast learning, reading complicated documentation,\ 
        creating clean data representation, data analysis],\
    benefits: [flexible hours, remote, health-insurance],\
    technology: [linux -medium, python -advanced, pytorch -advanced, matlabplot -beginner,\
    stable diffusion -beginner, vsc -advanced],\
    languages: [English C1, Polish C1]"""
    
jobOffer2 = """\
    "Tools developer (Javascript, Java)  Borealis Engineering Solutions  Category:  Fullstack ,\u00a0 JavaScript  Mid Must have Java\n JavaScript\n Design Patterns\n CI\n Git\n Maven\n Jenkins\n UI\n English (C2)Nice to have C\n C++\n Eclipse\n Requirements description \nBSc or MSc in Electrical Engineering or Computer Science or similar with 5+ years of experience in software development\nJava programming language knowledge\nJavascript programming language knowledge\nGood working knowledge of object-oriented design and design patterns\nStrong knowledge of with version control and CI tools (GIT, Maven, Jenkins).\nStrong analytical and communication skills\nGood comand in english, both written and oral communication\n\nThe following qualifications will be considered a plus:\n\nExperience with Vue.js and Quasar or other front end UI developer frameworks.\nExperience with Eclipse RCP (Rich Client Platform) development.\nFamiliar with Eclipse plug-in development across multiple operating systems\n\nThe following qualifications will be considered nice to have:\n\nC/C++ programming expertise\nKnowledge of embedded software development and debug practices.\nExperience working with open source communities and projects.\nKnowledge of PDE builds.\nExperience working in an Agile/Scrum environment\n Offer description Our partner is a key player in home automation, IoT and wireless technologies. As the leading provider of silicon, software, and solutions for a smarter, more connected world they offer full stack wireless connectivity technologies and a wide range of products. Budapest R&D center is tightly coupled with global research activities and integrated into the mainstream development areas.Your responsibilities In this role, you will create innovative software solutions and state-of-the-art development and hardware simulation tooling for multiple platforms.  \tDesign, implement, and unit test features and applications  \tUtilize innovative approaches to problem solving.  \tPerform design and code reviews  Job details\n Online recruitment   Recruitment language: Hungarian&English   Start ASAP   Permanent contract   Paid holiday for: Employment contract    Remote for 2 days a week   Flexible hours   Equipment supplied\n Computer: Notebook  Methodology\n Agile management Scrum, Kanban, Agile Issue tracking tool  Version control system GIT Build server Jenkins Perks in the office\n Free coffee  No dress code  In-house trainings  Benefits\n International projects  Small teams"""
    
jobOffer3 = """
    Junior JavaScript Developer  4 The Player  Category:  Frontend ,\u00a0 JavaScript  Junior Must have JavaScript\n HTML\n OOP\n Git\n C#\n TypeScript\n Polish  English Nice to have Webpack\n CSS\n Requirements description Optional:\n\nPixi.js\nTeam player\nCommunication skills\nnode.js\nhowler.js\nAttention to detail\nBeing able to deal with the pressure\nWilling to learn\n Offer description We are looking for an awesome JavaScript Game Developer to join our new remote games development team. 4ThePlayer.com is an up-and-coming creator of online gambling games (slots, table games and others). The company was founded in 2018 by two luminaries of the gambling gaming industry, Chris Ash and Andrew Porter, who have a combined 40 years of gaming industry experience. 4ThePlayer has been tipped for significant success, shortlisted to industry awards (EGR Awards) and rapidly increasing our reputation for a forward-thinking approach to game design. We are on track for exponential growth and to become an industry leader.Your responsibilities Creating games using high-quality code in JavaScript/HTML5  Implement animations, effects, and particle systems  Identify and resolve development problems  Collaborate with developers, graphic artists, audio and QA team  Job details\n Online recruitment   Recruitment language: English&Polish   Start ASAP   Permanent contract   Paid holiday for: B2B    Fully remote   Flexible hours   No travel involved   Mainly new features   Mo\u017cliwo\u015b\u0107 negocjacji wynagrodzenia: Tak   Equipment supplied\n Windows  Computer: As you wish :)  As you wish :)  Methodology\n Agile management Agile Issue tracking tool Jira, Trello, Github Knowledge repository GitHub Code reviews GitHub Version control system GIT Benefits\n Small teams  International projects  Flat structure  Paid holiday  Fridays slot session  Work in the international team  Professional growth"
 """
 
jobOffer4 = """AI Developer  lemon.io  Category:  AI ,\u00a0 Python  Senior Must have Python\n AI\n Java\n C++\n Machine Learning\n NLP\n TensorFlow\n PyTorch\n Keras\n Degree\n SAN\n English (B2) Requirements description \nMinimum of 3 years of proven experience in applying AI to practical technology solutions\nSolid understanding of common programming languages used in AI, such as Python, Java, C++, and R\nHands-on experience in machine learning/NLP/deep learning/computer vision\nExperience with TensorFlow/Caffe/PyTorch/Keras/Scikit Learn/Theano\nExperience working with large data sets and writing efficient code capable of processing large data streams at speed\nStrong analytical skills for designing and implementing machine learning models, analyzing data sets, and identifying patterns and trends\nFamiliarity with Cloud environments\nBachelor\u2019s/Master\u2019s degree in computer science, data science, software development, or another related field would be a plus\nCreative mindset to develop innovative solutions to complex problems and challenges in AI development\nProblem-solving skills to identify and solve complex problems using AI techniques\nCommitment to continuous learning and keeping up with the latest advancements in AI technology and techniques.\nGood command of English, both written and spoken, as you\u2019ll be communicating with clients directly\nStrong organizational skills \u2014 ability to work full-time remotely with no supervision\nResponsibility \u2014 we want to trust you\nSoft skills \u2014 we value clear and effective communication, at the same time don't force you becoming a public speaker\n\nReady to take your career to the next level? Apply now and join the Lemon.io family!\nIf your experience matches with our requirements be ready for the next steps:\n\nVideoAsk (about 10 minutes)\nCompleting your me.lemon profile\n30 minutes Screening call with our Recruiters\nTechnical Interview with our Developers\nFeedback\nMagic Box (we are looking for best project for you)\n\nP.S. We work with developers from LATAM, Canada, UK, Europe BUT we have some exceptions.\nAt the moment, we don\u2019t have a legal basis to accept applicants from Monaco, Belarus, Russia, Serbia, Denmark, Norway, Luxemburg, Iceland, Liechtenstein, San Marino, Belize, Cuba, Nicaragua, Saint Lucia, so we don\u2019t work with applicants from these countries. Offer description Are you a talented developer looking for a remote job that lets you showcase your skills while also enjoying better compensation and career growth? Look no further than Lemon.io, the marketplace that connects you with hand-picked startups in the US and Europe.\nYou'll love working with us, because:\n\nWe understand that respecting your time is key to your success, which is why we connect you with teams that won't micromanage or require time trackers.\nYou can earn $5k - $10k monthly\u00a0(the rate may differ due to skill and/or level of experience), and we've already paid out over $10M to our engineers.\nWith flexible working hours and the ability to work asynchronously, you can choose a schedule that works best for you.\nSay goodbye to project managers, and hello to direct communication with clients. Most of them have technical backgrounds. Sounds good, yeah?\nNo more hunting for clients or negotiating rates - let us handle the business side of things so you can focus on what you do best.\nEnjoy the fast-paced startup environment that will keep you motivated and engaged.\nPass the interviews now and start working when you're ready - even if that's a few months down the line.\nWe'll manually find you the best project to match your skills and preferences.\nConnect with the best developers in the world through our community.\n Job details\n Recruitment language: English   Start ASAP   Permanent contract   Paid holiday for: Employment contract    Fully remote   Flexible hours   Equipment supplied\n Linux  Apple  Windows  Methodology\n Agile management  Issue tracking tool"""

In [52]:
queryRank = """Please, process given job offer and in the answer put an \
    header: "Verdict: match-level", where match levels are:\
    [Not appropriate, Acceptable, Ok, Super]. Chosen level shall depend on this how well given job \
    offer matches client job qualifications and preferences.
    
    Instructions:
    If client's job qualifications and preferences are very close or identical to job's profile and requirements return 'Super'
    For example if you see that client is senior data scientist and job offer is related to senior data scientist return 'Super' 
    
    If client's job qualifications and preferences matches with at least half of job profile and requirements return 'Ok'.
    
    If client's job qualifications and preferences somewhat meet job's important requirements \
    and profile return 'Acceptable'. For example One third of job's requirements are met mean that you should return 'Acceptable'
    
    If client's job qualifications and preferences mostly do not meet job's important requirements \
    and profile return 'Not Appropriate'
    
    
    job offer: 
    {jobOffer}; \
    client's job qualifications and preferences:
    {personalData};\
    """

promptRank = PromptTemplate(template=queryRank, input_variables=["jobOffer", "personalData"])

chainRank = LLMChain(llm=llm, prompt=promptRank, output_key='Verdict')

In [53]:
desc = """
    You will be given 3 things:
    1. Job offer
    2. Client's job qualifications and preferences
    3. Recruiters' verdict ('Not appropriate' or 'Acceptable' or 'Ok' or 'Super')

    Describe a rationale behind recruiters' verdict. Find argument for why this verdict was given for this candidate and job offer. \
    Your description should be brief and should not exceed 3 sentences. Thy to make that decription more like a brief comment.
    
    If Recrutier's verdict is 'Not appropriate':
    Just say nothing, your response cannot contain any words and signs.
    
    If Recrutier's verdict is 'Acceptable':
    In brief words write a comment that will describe why job offer might be ok for clients qualifications and preferences.
    Do not exceed 3 sentences. Do not be too enthustiastic, mention what qualifications or preferences are not met. 
    Thy to make that decription more like a brief comment. Do not mention explicitly an information that job offer is Acceptable, because
    client already knows that.
    
    If Recrutier's verdict is 'Ok':
    In brief words write a comment that will describe why job offer is ok choice for clients qualifications and preferences.
    Do not exceed 3 sentences. Write what requirements are not met. Thy to make that decription more like a brief comment.
    Do not mention explicitly an information that job offer is Acceptable, because
    client already knows that.
    
    If Recrutier's verdict is 'Super':
    In brief words write a comment that will describe why job offer is ok choice for clients qualifications and preferences.
    Do not exceed 3 sentences. Write that that job is perfect for the client, be very enthustiastic. 
    Thy to make that decription more like a brief comment.
    Do not mention explicitly an information that job offer is Acceptable, because
    client already knows that.

    Job Offer:
    {jobOffer}
    Candidate's job qualifications and preferences:
    {personalData}
    Recruiters' verdict:
    {Verdict}
    """

desc_prompt = PromptTemplate(template=desc, input_variables=["jobOffer", "personalData", "Verdict"])

# desc_prompt = PromptTemplate(
#     template=desc_template,
#     input_variables=["Verdict", "jobOffer", "personalData"],
# )

chainDescription = LLMChain(llm=llm, prompt=desc_prompt, output_key='Description')

In [54]:
overall_chain = SequentialChain(
    chains=[chainRank, chainDescription],
    input_variables=["jobOffer", "personalData"],
    output_variables=["Verdict", "Description"],
    verbose=True
)

In [55]:
# result = overall_chain({'jobOffer': jobOffer, 'personalData': personalData})

# result['Verdict']
# result['Description']



> Entering new SequentialChain chain...

> Finished chain.


"This job offer is a perfect match for the candidate's qualifications and preferences. The candidate's skills in fast learning, reading complicated documentation, creating clean data representation, and data analysis align perfectly with the job requirements for a Data Scientist or AI Software Engineer. Additionally, the candidate's interests in generating AI images, data analysis, and large language models make them an ideal fit for this position."

In [56]:
# result2 = overall_chain({'jobOffer': jobOffer2, 'personalData': personalData})

# result2['Verdict']
# result2['Description']



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jt4dQuftpvIyXKTy70KOdMlC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jt4dQuftpvIyXKTy70KOdMlC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


> Finished chain.


"The job offer is acceptable for the client's qualifications and preferences. However, the client's preference for AI-related roles and interests in generating AI images and large language models are not fully met by the job offer for a Tools Developer position."

In [57]:
# result3 = overall_chain({'jobOffer': jobOffer3, 'personalData': personalData})

# result3['Verdict']
# result3['Description']



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jt4dQuftpvIyXKTy70KOdMlC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jt4dQuftpvIyXKTy70KOdMlC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


> Finished chain.


"The job offer for a Junior JavaScript Developer at 4 The Player is acceptable for the candidate's qualifications and preferences. However, the candidate's skills in Python, PyTorch, and data analysis may not be fully utilized in this role."

In [58]:
# result4 = overall_chain({'jobOffer': jobOffer4, 'personalData': personalData})

# result4['Verdict']
# result4['Description']



> Entering new SequentialChain chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jt4dQuftpvIyXKTy70KOdMlC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-jt4dQuftpvIyXKTy70KOdMlC on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


> Finished chain.


"The job offer is acceptable for the client's qualifications and preferences. The client meets the required qualifications in terms of Python and PyTorch skills, and their interests in data analysis align with the job requirements. However, their skills in Matlabplot and Stable Diffusion are not at an advanced level."

In [85]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import pandas as pd
import tiktoken

# class PageContent:
#     def __init__(self, content):
#         self.page_content = content
#         self.metadata = ""

# data = pd.read_json('../data/all_offers_with_desc_pruned.json', orient='index')['Description'].reset_index(drop=True).to_list()
# data_dics = [PageContent(content) for content in data]

# db = Chroma.from_documents(data_dics, OpenAIEmbeddings())

data = pd.read_json('../data/all_offers_with_desc_pruned.json', orient='index')['Description'].reset_index(drop=True)\
    .sample(frac=1).to_list()
data = data[:len(data)]
filtered_list = list(filter(None, data))


db = Chroma.from_texts(filtered_list, HuggingFaceEmbeddings())

In [59]:
raise

RuntimeError: No active exception to reraise

In [ ]:
result = chainRank({"jobOffer": jobOffer, "personalData": personalData})

In [ ]:
print(result)

In [ ]:
superVerdictTemplate = """You are given a job offer and client personal perferences and qualifications. \
    Your task is to briefly, reason why given job offer would be good choice for a client. Be very enthustiastic, \
    try to limit your answer to just two sentences. Write only that reasoning in your answer. \
    Here is job offer: 
    {jobOffer} 
    
    And here are informations about a client:
    {personalData}"""
    
okVerdictTemplate = """You are given a job offer and client personal perferences and qualifications. \
    Your task is to briefly, reason why given job offer would be ok choice for a client. \
    Try to limit your answer to just three sentences. Tell what might not suit a client in the offer. \
    Write only that reasoning in your answer. \
    Here is job offer: 
    {jobOffer} 
    
    And here are informations about a client:
    {personalData}"""
    
acceptableVerdictTemplate = """You are given a job offer and client personal perferences and qualifications. \
    Your task is to briefly, reason why given job offer would be ok choice for a client. \
    Try to limit your answer to just three sentences. Tell that job can be hard to get with the client's qualifications or \
    might just not suit the client's preferences. \
    Write only that reasoning in your answer. \
    Here is job offer: 
    {jobOffer} 
    
    And here are informations about a client:
    {personalData}"""

# def transform_func(inputs: dict) -> dict:
#     return {"description": ""} 

# default_chainsssss = TransformChain(
#     input_variables=["input"],
#     output_variables=["Reasoning"],
#     transform=transform_func
# )

default_chain = ConversationChain(llm=llm, output_key="Reasoning")

prompt_infos = [
    {
        "name": "Acceptable", 
        "description": "Good for describing acceptable job offers based on 2 variables: job_offer and personal_data", 
        "prompt_template": acceptableVerdictTemplate
    },
    {
        "name": "Ok", 
        "description": "Good for describing Ok job offers based on 2 variables: job_offer and personal_data", 
        "prompt_template": okVerdictTemplate
    },
    {
        "name": "Super", 
        "description": "Good for describing super job offers based on 2 variables: job_offer and personal_data", 
        "prompt_template": superVerdictTemplate
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=['personalData', 'jobOffer'])
    chain = LLMChain(llm=llm, prompt=prompt, output_key="Reasoning")
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
 
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
Verdict: {{Verdict}}
jobOffer: {{jobOffer}}
personalData: {{personalData}}

<< OUTPUT (remember to include the ```json)>>""" 
  
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)  
        
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["Verdict", "jobOffer", "personalData"],
    output_parser=RouterOutputParser(),
)

routerChainDesc = LLMRouterChain.from_llm(llm, router_prompt)

chainDescription = MultiPromptChain(router_chain=routerChainDesc, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain)

In [ ]:
overall_chain = SequentialChain(
    chains=[chainRank, chainDescription],
    input_variables=["jobOffer", "personalData"],
    output_variables=["Verdict", "text"],
    verbose=True
)

In [ ]:
overall_chain.chains

In [ ]:
import langchain


langchain.debug = True
result = overall_chain({"jobOffer": jobOffer, "personalData": personalData})
print(result)